<a href="https://colab.research.google.com/github/truongcan0512/Image-analysis/blob/main/Lab2_color_features_using_CH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!unzip /content/drive/MyDrive/leedsbutterfly_dataset_v1.0.zip

Archive:  /content/drive/MyDrive/leedsbutterfly_dataset_v1.0.zip
   creating: leedsbutterfly/descriptions/
  inflating: leedsbutterfly/descriptions/001.txt  
  inflating: leedsbutterfly/descriptions/002.txt  
  inflating: leedsbutterfly/descriptions/003.txt  
  inflating: leedsbutterfly/descriptions/004.txt  
  inflating: leedsbutterfly/descriptions/005.txt  
  inflating: leedsbutterfly/descriptions/006.txt  
  inflating: leedsbutterfly/descriptions/007.txt  
  inflating: leedsbutterfly/descriptions/008.txt  
  inflating: leedsbutterfly/descriptions/009.txt  
  inflating: leedsbutterfly/descriptions/010.txt  
   creating: leedsbutterfly/images/
  inflating: leedsbutterfly/images/0010001.png  
  inflating: leedsbutterfly/images/0010002.png  
  inflating: leedsbutterfly/images/0010004.png  
  inflating: leedsbutterfly/images/0010005.png  
  inflating: leedsbutterfly/images/0010006.png  
  inflating: leedsbutterfly/images/0010007.png  
  inflating: leedsbutterfly/images/0010008.png  
  in

In [3]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

In [4]:
# Buoc 1: Trich xuat object pixel
"""
Ta se chon ra 30 images de tinh color centers
Y tuong: 
- Voi moi object, ta tim ra nhung pixels khac khong bang cach tim
trong mask.
- Sau do, ung moi moi toa do x,y trong mask, ta match voi image -> obj pixels
"""
List_images = os.listdir("/content/leedsbutterfly/images") #random
num_images = len(List_images)
OBJ_pixels = np.zeros((1,3))

# Chon 30 images de tinh color centers

for i in range(0,30):
  imBGR = cv2.imread("/content/leedsbutterfly/images/" + List_images[i]) #Đọc ảnh, chỉ cần đưa tên đường dẫn ảnh vào -> ảnh BRG
  imRGB = cv2.cvtColor(imBGR, cv2.COLOR_BGR2RGB) #Chuyển ảnh về dạng RGB
  imRGB = imRGB/255 # Chuan hoa ve [0;1]

  seg_path = "/content/leedsbutterfly/segmentations/" + List_images[i].replace(".png","_seg0.png") # Cac goi file segmentation bang cach ".png"-->"_seg0.png"
  mask = cv2.imread(seg_path)
  mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)

  [x,y]=np.nonzero(mask[:,:,0]) # Ham nonzeros tra ve index cua cac phan tu khac 0 trong ma tran
  obj=imRGB[x,y,:] # Lay pixel cua object
  OBJ_pixels = np.concatenate((OBJ_pixels, obj), axis=0) # Noi cac obj theo hang


In [5]:
# Buoc 2: Quantization
"""
Muc dich: Dua anh ve chi con 16 mau chinh.
Y tuong:
- Ta chon ra mot tap con bao gom 10.000 pixels trong tong so OBJ_pixels
- Sau do, ta su dung clustering de phan cum du lieu ta thu duoc cac pixel
centers, ta thay cac pixels bang cac pixel centers --> quantization
"""

sub_image_array = shuffle(OBJ_pixels,n_samples=10000) # Chon ngau nhien 10.000 samples trong OBJ_pixels de xay dung model KMeans

# Build model using KMeans with 16 clusters
color_quantization_model = KMeans(n_clusters=16) # Build model
color_quantization_model.fit(sub_image_array) # Fit model

# Quantization image
quantization_img = color_quantization_model.cluster_centers_[color_quantization_model.predict(OBJ_pixels)] #Chuyen moi pixel thanh center cua no


In [6]:
# Buoc 3: Trich xuat dac trung cho anh, dua ve dang vector cua ML
"""
Muc dich: Voi moi anh, ta se xem xet so lan xuat hien cua moi centers
Y tuong:
- Su dung ham histogram trong thu vien numpy
"""
X=[] # feature vector includes 16 color centers
label=[] # label
for i in range(0,num_images):
  imBGR = cv2.imread("/content/leedsbutterfly/images/" + List_images[i]) #Đọc ảnh, chỉ cần đưa tên đường dẫn ảnh vào -> ảnh BRG
  imRGB = cv2.cvtColor(imBGR, cv2.COLOR_BGR2RGB) #Chuyển ảnh về dạng RGB
  imRGB = imRGB/255

  seg_path = "/content/leedsbutterfly/segmentations/" + List_images[i].replace(".png","_seg0.png") # Cac goi file segmentation bang cach ".png"-->"_seg0.png"
  mask = cv2.imread(seg_path)
  mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)

  [x,y]=np.nonzero(mask[:,:,0]) # Ham nonzeros tra ve toa do cua cac phan tu khac 0 trong ma tran
  obj=imRGB[x,y,:] # Lay pixel cua object
  color_id = color_quantization_model.predict(obj) #ma tran color_id bieu dien cac label cho moi sample

  # gia tri bins=so centers
  color_histogram = np.histogram(color_id, bins=16)[0]/(np.shape(obj)[0]*np.shape(obj)[1]) #Tinh toan so lan xuat hien của moi label va chuan hoa
  X.append(color_histogram)

  label.append(int(List_images[i][0:3]))


In [7]:
# Chia du lieu thanh training/test (chon 15 images/class de test)
# loai tru 30 images dau tien, va training phan con lai
# Thu lai KNN, SVM ==> Acc?
# Thay doi n_clustering ==> Acc?
# Color Moment ==> Accc?

In [8]:
# Buoc 4: Tao training/test

# Loai bo 30 phan tu dau tien 
X = np.array(X)
y = np.array(label)
X_data = X[30:]
y_data = y[30:]

# Lay moi class 15 images lam test
X_test = []
y_test = []
X_train = []
y_train = []
count = 0
for i in range(1,11):
  tmp = 0
  for j in range(0,np.shape(y_data)[0]):
    if(y_data[j] == i and tmp < 15):
      X_test.append(X_data[j])
      y_test.append(y_data[j])
      tmp = tmp+1
    else:
      if(count < np.shape(y_data)[0]):
        y_train.append(y_data[j])
        X_train.append(X_data[j])
      count = count + 1

X_test = np.array(X_test)
y_test = np.array(y_test)
X_train = np.array(X_train)
y_train = np.array(y_train)


In [9]:
knn_model=KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train,y_train)
knn_test=knn_model.predict(X_test)
print("Acc of KNN: ", accuracy_score(y_test, knn_test))

svm_model = SVC(C=999999, kernel="rbf", gamma=2, probability=True)
svm_model.fit(X_train, y_train)
print("Acc of SVM: ", svm_model.score(X_test, y_test))


Acc of KNN:  0.7733333333333333
Acc of SVM:  0.9933333333333333
